In [46]:
import pickle
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
  
lemmatizer = WordNetLemmatizer()
import re
def cleaning(sentence):
    stop_words = set(stopwords.words('english'))
    tokens = re.sub(r'[^\w\s]', '', sentence.lower()).replace("\n", " ").split(" ")
    cleaned = [token for token in tokens if token not in stop_words]
    return " ".join(cleaned)
labels = ["insurance-etc","investment", "medical-sales", "phising", "sexual", "software-sales"]
text = []
classes = []
for label in labels:
    path = "/Users/lorraine/Desktop/Spam_Filter/Annotated/"+label
  
    os.chdir(path)
  
    def read_text_file(file_path):
        with open(file_path, 'r', encoding='ISO-8859-1') as f:
            return f.read()
      
    for file in os.listdir():
    
        if file.endswith(".txt"):
            file_path = f"{path}/{file}"
            text.append(cleaning(str(read_text_file(file_path))))
            classes.append(label)

data = pd.DataFrame({'sentence':text, 'label':classes})
data

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lorraine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lorraine/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lorraine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,sentence,label
0,subject take advantage low interest rates no...,insurance-etc
1,subject lowest rate us history sound let 6000...,insurance-etc
2,subject application pre approved monday oct 1...,insurance-etc
3,subject final offer wed 29 sep 2004 19 16 0...,insurance-etc
4,subject application confirmation sun 14 nov 2...,insurance-etc
...,...,...
2177,subject popular software low low prices bury ...,software-sales
2178,subject clean ur computer 3 ey 85 chance comp...,software-sales
2179,subject software incredibly low prices 73 lo...,software-sales
2180,subject latest qem software available low pri...,software-sales


In [47]:
def tfidf(word):
    sentence = data['sentence']
    idf = np.log(len(sentence)/sentence.str.contains(word).sum())
    result = []
    for i in range(len(sentence)):
        tf = sentence.iloc[i].count(word)/(len(sentence.iloc[i]))
        result.append(tf*idf)
    return result


In [48]:
import json
f = open('/Users/lorraine/Desktop/Spam_Filter/seedwords.json')
seeds = json.load(f)
result = pd.DataFrame()
for key, value in seeds.items():
    df = pd.DataFrame()
    for w in value:
        df[w] = tfidf(w)
    result[key] = df.sum(axis = 1)
result


/var/folders/gr/mx83vpvn7z713f1dzv81v63r0000gn/T/ipykernel_25696/3118394667.py:3: RuntimeWarning: divide by zero encountered in long_scalars
  idf = np.log(len(sentence)/sentence.str.contains(word).sum())
/var/folders/gr/mx83vpvn7z713f1dzv81v63r0000gn/T/ipykernel_25696/3118394667.py:7: RuntimeWarning: invalid value encountered in double_scalars
  result.append(tf*idf)
/var/folders/gr/mx83vpvn7z713f1dzv81v63r0000gn/T/ipykernel_25696/3118394667.py:3: RuntimeWarning: divide by zero encountered in long_scalars
  idf = np.log(len(sentence)/sentence.str.contains(word).sum())
/var/folders/gr/mx83vpvn7z713f1dzv81v63r0000gn/T/ipykernel_25696/3118394667.py:7: RuntimeWarning: invalid value encountered in double_scalars
  result.append(tf*idf)
/var/folders/gr/mx83vpvn7z713f1dzv81v63r0000gn/T/ipykernel_25696/3118394667.py:3: RuntimeWarning: divide by zero encountered in long_scalars
  idf = np.log(len(sentence)/sentence.str.contains(word).sum())
/var/folders/gr/mx83vpvn7z713f1dzv81v63r0000gn/T/ipyk

,insurance-etc,investment,medical-sales,phising,sexual,software-sales
0,0.032397,0.012083,0.000000,0.000000,0.007124,0.000000
1,0.057250,0.003857,0.000000,0.000000,0.000000,0.000000
2,0.015594,0.000000,0.001352,0.000000,0.000000,0.000000
3,0.012809,0.006285,0.000000,0.003700,0.000000,0.017154
4,0.016872,0.000000,0.000000,0.000000,0.001902,0.000000
...,...,...,...,...,...,...
2177,0.000000,0.003636,0.000000,0.000000,0.000000,0.003636
2178,0.000000,0.000000,0.003091,0.000000,0.000000,0.000000
2179,0.000000,0.003785,0.002142,0.000000,0.000000,0.003785
2180,0.000000,0.002509,0.000000,0.010853,0.000000,0.051350


In [49]:
data["prediction"] = result.idxmax(1)
data

,sentence,label,prediction
0,subject take advantage low interest rates no...,insurance-etc,insurance-etc
1,subject lowest rate us history sound let 6000...,insurance-etc,insurance-etc
2,subject application pre approved monday oct 1...,insurance-etc,insurance-etc
3,subject final offer wed 29 sep 2004 19 16 0...,insurance-etc,software-sales
4,subject application confirmation sun 14 nov 2...,insurance-etc,insurance-etc
...,...,...,...
2177,subject popular software low low prices bury ...,software-sales,investment
2178,subject clean ur computer 3 ey 85 chance comp...,software-sales,medical-sales
2179,subject software incredibly low prices 73 lo...,software-sales,investment
2180,subject latest qem software available low pri...,software-sales,software-sales


In [50]:
# micro and macro F1 using tf-idf
from sklearn import metrics

In [51]:
metrics.f1_score(data["label"], data["prediction"], average="micro")

0.6585701191567369

In [52]:
metrics.f1_score(data["label"], data["prediction"], average="macro")

0.6326503276247037

In [53]:

import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
def preprocessing(sentence):
    tokens = sentence.split(" ")
    return [token for token in tokens if token!="" and token != " "]
features = data["sentence"].apply(preprocessing)
model = Word2Vec(sentences=features, size=100, window=5, min_count=1, workers=4)
#features

In [54]:
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
model.train(features, total_examples=len(data), epochs=20)
#vector = model.wv["atheism"]
#vector

(6195351, 6290560)

In [60]:
def get_vectors_per_label(filename):
    f = open(filename)
    seeds = json.load(f)
    vector_per_label = []
    for key, value in seeds.items():
        lst = []
        for w in value:
            lst.append(model.wv[w])
        arr = np.asarray(lst)
        total = np.average(arr, axis=0)
        vector_per_label.append(total)
    return vector_per_label
vector_per_label = get_vectors_per_label('/Users/lorraine/Desktop/Spam_Filter/seedwords.json')

In [61]:
def get_vector_per_doc(feature):
    vector_per_doc = []
    for feat in feature:
        lst = []
        for w in feat:
            lst.append(model.wv[w])
        arr = np.asarray(lst)
        total = np.average(arr, axis=0)
        vector_per_doc.append(total)
    return vector_per_doc
vector_per_doc = get_vector_per_doc(features)

In [62]:
len(vector_per_doc)

2182

In [64]:
f = open('/Users/lorraine/Desktop/Spam_Filter/seedwords.json')
seeds = json.load(f)
from numpy.linalg import norm
def predict_word2vec(vector_per_doc, vector_per_label):
    predictions = []
    labels = list(seeds.keys())
    for doc in vector_per_doc:
        cosine = []
        for label in vector_per_label:
            cosine.append(np.dot(doc,label)/(norm(doc)*norm(label)))
        max_value = max(cosine)
        max_index = cosine.index(max_value)
        predictions.append(labels[max_index])
    return predictions   
prediction_word2vec = predict_word2vec(vector_per_doc, vector_per_label)

In [65]:
data["prediction_word2vec"] = prediction_word2vec
data

,sentence,label,prediction,prediction_word2vec
0,subject take advantage low interest rates no...,insurance-etc,insurance-etc,insurance-etc
1,subject lowest rate us history sound let 6000...,insurance-etc,insurance-etc,insurance-etc
2,subject application pre approved monday oct 1...,insurance-etc,insurance-etc,sexual
3,subject final offer wed 29 sep 2004 19 16 0...,insurance-etc,software-sales,insurance-etc
4,subject application confirmation sun 14 nov 2...,insurance-etc,insurance-etc,insurance-etc
...,...,...,...,...
2177,subject popular software low low prices bury ...,software-sales,investment,medical-sales
2178,subject clean ur computer 3 ey 85 chance comp...,software-sales,medical-sales,software-sales
2179,subject software incredibly low prices 73 lo...,software-sales,investment,medical-sales
2180,subject latest qem software available low pri...,software-sales,software-sales,software-sales


In [66]:
# micro and macro F1 using word2vec
metrics.f1_score(data["label"], data["prediction_word2vec"], average="micro")

0.576076993583868

In [67]:
metrics.f1_score(data["label"], data["prediction_word2vec"], average="macro")

0.5703324192605602

In [68]:
sum(data["prediction_word2vec"] == data["label"])/len(data)

0.576076993583868